In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# Cargar datos
df = pd.read_csv('data.csv', parse_dates=['fecha'], dayfirst=True)

# Ingeniería de características temporales
df['dia_del_año'] = df['fecha'].dt.dayofyear
df['sin_dia'] = np.sin(2 * np.pi * df['dia_del_año'] / 365.25)
df['cos_dia'] = np.cos(2 * np.pi * df['dia_del_año'] / 365.25)

# Transformar a problema de clasificación multiclase
def clasificar_precipitacion(precip):
    if precip == 0:
        return 0  # Nula
    elif precip <= 2:
        return 1  # Leve
    elif precip <= 15:
        return 2  # Moderada
    else:
        return 3  # Intensa

df['clase_precipitacion'] = df['precipitacion_total'].apply(clasificar_precipitacion)

# Preparar características
features = ['temp_max', 'temp_min', 'presion_max', 'presion_min', 
            'rocio_max', 'rocio_min', 'viento_promedio', 'radiacion', 
            'sin_dia', 'cos_dia']
X = df[features]
y = df['clase_precipitacion']

# Escalado y división
scaler = StandardScaler()
X_escalado = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_escalado, y, test_size=0.3, random_state=42)

# Definir modelos
modelos = {
    'SVM': SVC(kernel='rbf', C=1.0, probability=True, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='mlogloss'),
    'Naive Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# Entrenar y evaluar modelos
resultados = {}
for nombre, modelo in modelos.items():
    # Entrenamiento
    modelo.fit(X_train, y_train)
    
    # Predicciones
    y_pred = modelo.predict(X_test)
    y_pred_proba = modelo.predict_proba(X_test) if hasattr(modelo, "predict_proba") else None
    
    # Almacenar resultados
    resultados[nombre] = {
        'modelo': modelo,
        'predicciones': y_pred,
        'probabilidades': y_pred_proba
    }
    print(f"{nombre} entrenado correctamente")

# Los resultados están disponibles en el diccionario 'resultados'

# Comparación final de modelos
print("\n" + "="*80)
print("COMPARACIÓN FINAL DE MODELOS")
print("="*80)

# Crear DataFrame para comparación
comparacion = []
for nombre, metrics in resultados.items():
    comparacion.append({
        'Modelo': nombre,
        'Accuracy': f"{metrics['accuracy']:.4f}",
        'Precision': f"{metrics['precision']:.4f}",
        'Recall': f"{metrics['recall']:.4f}",
        'F1-Score': f"{metrics['f1']:.4f}",
        'Mejores Parámetros': str(metrics['mejores_parametros'])[:50] + "..." if len(str(metrics['mejores_parametros'])) > 50 else metrics['mejores_parametros']
    })

df_comparacion = pd.DataFrame(comparacion)
print(df_comparacion.to_string(index=False))

# Mejor modelo basado en F1-Score (balance entre precision y recall)
mejor_modelo_nombre = max(resultados.items(), key=lambda x: x[1]['f1'])[0]
mejor_modelo = resultados[mejor_modelo_nombre]['modelo']

print(f"\n MEJOR MODELO: {mejor_modelo_nombre} (F1-Score: {resultados[mejor_modelo_nombre]['f1']:.4f})")

# Reporte de clasificación detallado del mejor modelo
print(f"\nREPORTE DE CLASIFICACIÓN - {mejor_modelo_nombre}:")
print(classification_report(y_test, resultados[mejor_modelo_nombre]['predicciones']))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Cargar datos
#df = pd.read_csv('data.csv', parse_dates=['fecha'], dayfirst=True)
df = pd.read_csv('dic1985-oct2025.csv', parse_dates=['fecha'])

# Ingeniería de características temporales
df['dia_del_año'] = df['fecha'].dt.dayofyear
df['sin_dia'] = np.sin(2 * np.pi * df['dia_del_año'] / 365.25)
df['cos_dia'] = np.cos(2 * np.pi * df['dia_del_año'] / 365.25)

# Transformar a problema de clasificación multiclase
def clasificar_precipitacion(precip):
    if precip == 0:
        return 0  # Nula
    elif precip <= 2:
        return 1  # Leve
    elif precip <= 15:
        return 2  # Moderada
    else:
        return 3  # Intensa

df['clase_precipitacion'] = df['precipitacion_total'].apply(clasificar_precipitacion)

# Preparar características
features = ['temp_max', 'temp_min', 'presion_max', 'presion_min', 
            'rocio_max', 'rocio_min', 'viento_promedio', 'radiacion', 
            'sin_dia', 'cos_dia']
X = df[features]
y = df['clase_precipitacion']

# Escalado y división
scaler = StandardScaler()
X_escalado = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_escalado, y, test_size=0.3, random_state=42)

# Definir modelos
modelos = {
    'SVM': SVC(kernel='rbf', C=1.0, probability=True, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='mlogloss'),
    'Naive Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# Entrenar y evaluar modelos
resultados = {}
metricas_comparativas = []

print("=== ENTRENAMIENTO DE MODELOS ===")
for nombre, modelo in modelos.items():
    # Entrenamiento
    modelo.fit(X_train, y_train)
    
    # Predicciones
    y_pred = modelo.predict(X_test)
    y_pred_proba = modelo.predict_proba(X_test) if hasattr(modelo, "predict_proba") else None
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    # Almacenar resultados
    resultados[nombre] = {
        'modelo': modelo,
        'predicciones': y_pred,
        'probabilidades': y_pred_proba,
        'metricas': {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    }
    
    # Guardar para comparación
    metricas_comparativas.append({
        'Modelo': nombre,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print(f"✅ {nombre} entrenado correctamente")

# Crear DataFrame comparativo
df_comparativo = pd.DataFrame(metricas_comparativas)
df_comparativo = df_comparativo.round(4)

print("\n" + "="*80)
print("COMPARACIÓN DE MÉTRICAS POR MODELO")
print("="*80)
print(df_comparativo.to_string(index=False))

# Encontrar el mejor modelo basado en F1-Score
mejor_modelo_info = df_comparativo.loc[df_comparativo['F1-Score'].idxmax()]
mejor_modelo_nombre = mejor_modelo_info['Modelo']
mejor_modelo = resultados[mejor_modelo_nombre]['modelo']

print("\n" + "="*80)
print("🏆 MEJOR MODELO SELECCIONADO")
print("="*80)
print(f"Modelo: {mejor_modelo_nombre}")
print(f"F1-Score: {mejor_modelo_info['F1-Score']:.4f}")
print(f"Accuracy: {mejor_modelo_info['Accuracy']:.4f}")
print(f"Precision: {mejor_modelo_info['Precision']:.4f}")
print(f"Recall: {mejor_modelo_info['Recall']:.4f}")

# Mostrar reporte detallado del mejor modelo
print(f"\n📊 REPORTE DETALLADO DEL MEJOR MODELO ({mejor_modelo_nombre})")
print("="*80)
y_pred_mejor = resultados[mejor_modelo_nombre]['predicciones']
print(classification_report(y_test, y_pred_mejor, 
                          target_names=['Nula', 'Leve', 'Moderada', 'Intensa']))

# Mostrar distribución de clases real vs predicha
print("\n📈 DISTRIBUCIÓN DE CLASES")
print("="*80)
dist_real = y_test.value_counts().sort_index().values
dist_pred = pd.Series(y_pred_mejor).value_counts().sort_index().values
df_dist = pd.DataFrame({
    'Real': dist_real,
    'Predicho': dist_pred
}, index=['Nula', 'Leve', 'Moderada', 'Intensa'])
print(df_dist)

# Ordenar modelos por F1-Score para ranking
print("\n🏅 RANKING DE MODELOS (por F1-Score)")
print("="*80)
df_ranking = df_comparativo.sort_values('F1-Score', ascending=False).reset_index(drop=True)
df_ranking['Posición'] = df_ranking.index + 1
df_ranking = df_ranking[['Posición', 'Modelo', 'F1-Score', 'Accuracy', 'Precision', 'Recall']]
print(df_ranking.to_string(index=False))

# El mejor modelo queda disponible para uso futuro
print(f"\n💡 El mejor modelo '{mejor_modelo_nombre}' está listo para hacer predicciones")
print("   Puedes usarlo como: mejor_modelo.predict(nuevos_datos)")

=== ENTRENAMIENTO DE MODELOS ===
✅ SVM entrenado correctamente
✅ Decision Tree entrenado correctamente
✅ Random Forest entrenado correctamente
✅ XGBoost entrenado correctamente
✅ Naive Bayes entrenado correctamente
✅ KNN entrenado correctamente

COMPARACIÓN DE MÉTRICAS POR MODELO
       Modelo  Accuracy  Precision  Recall  F1-Score
          SVM    0.7320     0.7311  0.7320    0.7192
Decision Tree    0.6315     0.6299  0.6315    0.6306
Random Forest    0.7210     0.7089  0.7210    0.7103
      XGBoost    0.7240     0.7144  0.7240    0.7169
  Naive Bayes    0.6246     0.6443  0.6246    0.6276
          KNN    0.6832     0.6696  0.6832    0.6728

🏆 MEJOR MODELO SELECCIONADO
Modelo: SVM
F1-Score: 0.7192
Accuracy: 0.7320
Precision: 0.7311
Recall: 0.7320

📊 REPORTE DETALLADO DEL MEJOR MODELO (SVM)
              precision    recall  f1-score   support

        Nula       0.83      0.90      0.86      2342
        Leve       0.57      0.52      0.54      1148
    Moderada       0.63      0.64

In [6]:
print(type(pd.Series(y_pred_mejor)))
print(type(y_test))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [7]:
print(pd.Series(y_pred_mejor).value_counts())

0    2522
1    1062
2     781
3       1
Name: count, dtype: int64


In [8]:
print(y_test.value_counts())

clase_precipitacion
0    2342
1    1148
2     772
3     104
Name: count, dtype: int64


In [15]:
# Mostrar distribución de clases real vs predicha
print("\n📈 DISTRIBUCIÓN DE CLASES")
print("="*80)
dist_real = y_test.value_counts().sort_index().values
dist_pred = pd.Series(y_pred_mejor).value_counts().sort_index().values
df_dist = pd.DataFrame({
    'Real': dist_real,
    'Predicho': dist_pred
}, index=['Nula', 'Leve', 'Moderada', 'Intensa'])
print(df_dist)


📈 DISTRIBUCIÓN DE CLASES
          Real  Predicho
Nula      2342      2522
Leve      1148      1062
Moderada   772       781
Intensa    104         1


In [16]:
import pickle
# save the classifier
with open(f'{mejor_modelo_nombre}.pkl', 'wb') as fid:
    pickle.dump(mejor_modelo, fid)   